In [7]:
# for setting-up, need to set-up nodejs because jupyterlab get extensions from nmp packages
# conda install -c conda-forge nodejs
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
# (may need to restart your lab)

In [8]:
!jupyter labextension list
!jupyter nbextension list

JupyterLab v2.2.8
Known labextensions:


   app dir: C:\Users\nguye\anaconda3\envs\sem-37\share\jupyter\lab
        @jupyter-widgets/jupyterlab-manager v2.0.0 enabled  ok
        jupyter-webrtc v0.5.0 enabled  ok


Known nbextensions:
  config dir: C:\Users\nguye\.jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 
  config dir: C:\Users\nguye\anaconda3\envs\sem-37\etc\jupyter\nbconfig
    notebook section
      jupyter-webrtc/extension enabled 
      jupyter-js-widgets/extension enabled 


      - Validating: ok
      - Validating: ok
      - Validating: ok


In [11]:
import ipywidgets
from ipywidgets import interact, interactive, Video
from ipywebrtc.webrtc import VideoStream
ipywidgets.__version__

'7.5.1'

In [7]:
video = Video.from_file('./data/small_videos/1.1.1_C1_trim.mp4')
video

Video(value=b'\x00\x00\x00 ftypmp42\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x0c;b mdat\x00\x00\x02…

In [28]:
from load_and_draw import load_and_draw, logger

logger.info('Input video: data/small_videos/6.2.5_C1_trim.mp4')
logger.info('Input segmentation: database.200731.1.csv')
logger.info('Output dir: output/')
load_and_draw(input_video_path='data/small_videos/6.2.5_C1_trim.mp4',
              input_segmentation='database.200731.1.csv', output_dir='output',
              output_video_name='output_video.mp4', region_length=60)

load_and_draw - INFO - Input video: data/small_videos/6.2.5_C1_trim.mp4
load_and_draw - INFO - Input segmentation: database.200731.1.csv
load_and_draw - INFO - Output dir: output/


In [10]:
# currently, video file created by opencv cannot be opened
output_video = Video.from_file('./output/output_video.mp4')
output_video

Video(value=b'\x00\x00\x00\x1cftypisom\x00\x00\x02\x00isomiso2mp41\x00\x00\x00\x08free\x06`\x89Jmdat\x00\x00\x…

In [13]:
# currently, video file created by opencv cannot be opened
output_video = VideoStream.from_file('./output/output_video.mp4')
output_video

VideoStream(video=Video(value=b'\x00\x00\x00\x1cftypisom\x00\x00\x02\x00isomiso2mp41\x00\x00\x00\x08free\x06`\…

In [3]:
def f(x):
    return x
interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f>

In [8]:
sl = widgets.IntSlider()
sl

IntSlider(value=0)